In [ ]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# PERIODO

In [ ]:
start_date = '2022-07-01'
final_date = '2022-07-29'

# DB

In [ ]:
db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2022\AMEX JULIO 2022.csv'
                   ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
                           ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
                           ,'MCC':object})


In [ ]:
db = pd.concat([db_1])
db.columns = db.columns.str.strip().str.lower() 
db['order_id'] = db['order_id'].str.strip()
db['order_id'] = db['order_id'].str.upper()
db['date_transaction'] = pd.to_datetime(db['date_transaction'])
db['INDEX_TOTAL'] = (db['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['amount'].astype(str).replace(' ', '',regex=True))
db['INDEX_PARCIAL'] = (db['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['amount'].astype(str).replace(' ', '',regex=True))
db['index_comisiones'] = (db['affiliation_type'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['msi'].str[:-6].astype(str).replace(' ', '',regex=True))
db['order_id'].fillna('SIN ORDER_ID EN DB',inplace=True)
db.head()

In [ ]:
db.shape

# EPA

In [ ]:
allfiles = glob.glob(r'G:\My Drive\EPA\EPA files\2022*\**\*.txt', recursive=True)
allfiles

In [ ]:
names_header=["1","2","3","4","5","6","7","8","9","10","A","12","13","14","15","16","17","18","19","order_id",
              "21","22","23","24","25","26","27","28"]

In [ ]:
epa = pd.concat([pd.read_csv(f,names=names_header, sep=',',dtype=object) for f in allfiles])
epa['order_id'] = epa['order_id'].str.strip()
epa.head()

In [ ]:
epa.shape

In [ ]:
epa['6'].unique()

In [ ]:
epa_0 = epa[(epa['6'] == '0')]
epa_1 = epa[(epa['6'] == '1')]
epa_3 = epa[(epa['6'] == '3')]
epa_4 = epa[(epa['6'] == '4')]
epa_5 = epa[(epa['6'] == '5')]
epa_9 = epa[(epa['6'] == '9')]

## EPA 0

In [ ]:
epa_0.head()

In [ ]:
epa_0.shape

## EPA 1

In [ ]:
epa_1['2'] = pd.to_datetime(epa_1['2'], format='%Y%m%d').dt.date
epa_1['2'] = pd.to_datetime(epa_1['2'])
epa_1.sort_values(by=['2'],inplace=True)
epa_1.head()

In [ ]:
epa_1.shape

In [ ]:
epa_1_final = epa_1[['6','2','8','15','16','17','18','19']]
epa_1_final.head()

In [ ]:
epa_1_final['8'] = epa_1_final['8'].astype(float)
epa_1_final['15'] = epa_1_final['15'].astype(float)
epa_1_final['16'] = epa_1_final['16'].astype(float)
epa_1_final['17'] = epa_1_final['17'].astype(float)
epa_1_final['18'] = epa_1_final['18'].astype(float)
epa_1_final['19'] = epa_1_final['19'].astype(float)

In [ ]:
epa_1_final.rename(columns ={'6':'Type','2':'Payment_date'},inplace=True)

In [ ]:
epa_1_final['Payment_amount'] = epa_1_final['8']/100
epa_1_final['Gross_amount'] = epa_1_final['15']/100
epa_1_final['Discount_amount'] = epa_1_final['16']/100
epa_1_final['Taxes_amount'] = epa_1_final['17']/100
epa_1_final['Discount_aceleration_amount'] = epa_1_final['18']/100
epa_1_final['Net_amount'] = epa_1_final['19']/100
epa_1_final.head()

In [ ]:
epa_1_excel = epa_1_final[(epa_1_final['Payment_date'] >= start_date) & (epa_1_final['Payment_date'] <= final_date)]

In [ ]:
epa_1_excel_final = epa_1_excel.groupby(['Payment_date']).agg({'Payment_amount':sum,'Gross_amount':sum,'Discount_amount':sum
                                                        ,'Taxes_amount':sum,'Discount_aceleration_amount':sum
                                                        ,'Net_amount':sum}).reset_index()
epa_1_excel_final.head()

In [ ]:
epa_1_excel.shape

## EPA 3

In [ ]:
epa_3['2'] = pd.to_datetime(epa_3['2'], format='%Y%m%d').dt.date
epa_3['2'] = pd.to_datetime(epa_3['2'])
epa_3.sort_values(by=['2'],inplace=True)
epa_3.head()

In [ ]:
epa_3.shape

In [ ]:
epa_3_final = epa_3[['6','2','10','A','12','13','14','15','16','25','26','27','28']]
epa_3_final.reset_index(drop=True, inplace=True)
epa_3_final.head()

In [ ]:
epa_3_final['10'] = epa_3_final['10'].astype(float)
epa_3_final['A'] = epa_3_final['A'].astype(float)
epa_3_final['12'] = epa_3_final['12'].astype(float)
epa_3_final['13'] = epa_3_final['13'].astype(float)
epa_3_final['14'] = epa_3_final['14'].astype(float)
epa_3_final['15'] = epa_3_final['15'].astype(float)
epa_3_final['16'] = epa_3_final['16'].astype(float)
epa_3_final['25'] = epa_3_final['25'].astype(float)
epa_3_final['26'] = epa_3_final['26'].astype(float)
epa_3_final['27'] = epa_3_final['27'].astype(float)
epa_3_final['28'] = epa_3_final['28'].astype(float)

In [ ]:
epa_3_final.rename(columns ={'6':'Type','2':'Payment_date','16':'Register_number'},inplace=True)

In [ ]:
epa_3_final['Gross_amount'] = epa_3_final['10']/100
epa_3_final['Gross_amount_amex'] = epa_3_final['A']/100
epa_3_final['Discount_amount'] = epa_3_final['12']/100
epa_3_final['Taxes_amount'] = epa_3_final['13']/100
epa_3_final['Discount_aceleration_amount'] = epa_3_final['14']/100
epa_3_final['Net_amount'] = epa_3_final['15']/100
epa_3_final['Aceleration_net_amount'] = epa_3_final['25']/100
epa_3_final['Gross_amount_trans_cambio'] = epa_3_final['26']/100
epa_3_final['Credit_gross_amount'] = epa_3_final['27']/100
epa_3_final['Unknown'] = epa_3_final['28']/100
epa_3_final.head()

In [ ]:
epa_3_excel = epa_3_final[(epa_3_final['Payment_date'] >= start_date) & (epa_3_final['Payment_date'] <= final_date)]

In [ ]:
epa_3_excel_final = epa_3_excel.groupby(['Payment_date']).agg({'Register_number':sum,'Gross_amount':sum,'Gross_amount_amex':sum
                                                               ,'Discount_amount':sum,'Taxes_amount':sum
                                                               ,'Discount_aceleration_amount':sum,'Net_amount':sum
                                                               ,'Aceleration_net_amount':sum
                                                               ,'Gross_amount_trans_cambio':sum,'Credit_gross_amount':sum
                                                               ,'Unknown':sum}).reset_index()
epa_3_excel_final.head()

In [ ]:
epa_3_excel_final.shape

## EPA 4

In [ ]:
epa_4['8'] = pd.to_datetime(epa_4['8'], format='%Y%m%d').dt.date
epa_4['2'] = pd.to_datetime(epa_4['2'], format='%Y%m%d').dt.date

In [ ]:
epa_4['8'] = pd.to_datetime(epa_4['8'])
epa_4['2'] = pd.to_datetime(epa_4['2'])

In [ ]:
epa_4.head()

In [ ]:
epa_4.shape

In [ ]:
txn_epa = epa_4[['order_id','6','8','A','12','22','23','2','15']]
txn_epa["12"] = txn_epa['12'].astype(float)
txn_epa["22"] = txn_epa['22'].astype(float)
txn_epa["23"] = txn_epa['23'].astype(float)


In [ ]:
txn_epa['total_amount_bruto'] = txn_epa['12']/100
txn_epa['fee_amount_mdr'] = -(txn_epa['total_amount_bruto']*0.0240)
txn_epa['fee_amount_msi'] = txn_epa['22']/100
txn_epa['iva_msi'] = txn_epa['fee_amount_msi']*0.1600
txn_epa['iva_mdr'] = txn_epa['fee_amount_mdr']*0.1600
txn_epa['bin_bank'] = txn_epa['A'].str[:6].astype(str)
txn_epa['last4_bank'] = txn_epa['A'].str[-8:-4].astype(str)

In [ ]:
txn_epa.rename(columns ={'6':'Type','8':'Date_process_roc','12':'Total_amount_charge_roc',
                         '22':'Svc_fee_amount_roc','23':'Svc_fee_amount_accel_roc','2':'roc_deposit_date'},inplace=True)
txn_epa['INDEX_TOTAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
# txn_epa['INDEX_PARCIAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['codigo_aprobacion'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
txn_epa.fillna('SIN ORDER_ID EN EPA',inplace=True)
txn_epa.head()

In [ ]:
txn_epa.shape

## EPA 5

In [ ]:
epa_5['2'] = pd.to_datetime(epa_5['2'], format='%Y%m%d').dt.date

In [ ]:
epa_5['2'] = pd.to_datetime(epa_5['2'])

In [ ]:
epa_5.head()

In [ ]:
epa_5.shape

In [ ]:
txn_epa_adj = epa_5[['2','8','9','10','A','12','13','14','15','16','18']]
txn_epa_adj.rename(columns ={'2':'roc_deposit_date','8':'clave_autorizacion','14':'no_tarjeta','15':'adj_id','16':'motivo_adj'},inplace=True) 
txn_epa_adj["9"] = txn_epa_adj['9'].astype(float)
txn_epa_adj["10"] = txn_epa_adj['10'].astype(float)
txn_epa_adj["A"] = txn_epa_adj['A'].astype(float)
txn_epa_adj["12"] = txn_epa_adj['12'].astype(float)
txn_epa_adj["13"] = txn_epa_adj['13'].astype(float)


In [ ]:
txn_epa_adj['importe_adj_bruto'] = txn_epa_adj['9']/100
txn_epa_adj['tasa_adj'] = txn_epa_adj['10']/100
txn_epa_adj['importe_impuesto'] = txn_epa_adj['A']/100
txn_epa_adj['tasa_servicio_plan_n'] = txn_epa_adj['12']/100
txn_epa_adj['importe_adj_neto'] = txn_epa_adj['13']/100
txn_epa_adj['bin_bank'] = txn_epa_adj['no_tarjeta'].str[:6].astype(str)
txn_epa_adj['last4_bank'] = txn_epa_adj['no_tarjeta'].str[-8:-4].astype(str)


In [ ]:
txn_epa_adj.head()

In [ ]:
txn_epa_adj.shape

In [ ]:
txn_epa_adj_final = txn_epa_adj[(txn_epa_adj['roc_deposit_date'] >= start_date) & (txn_epa_adj['roc_deposit_date'] <= final_date)]

## EPA 9

In [ ]:
epa_9.head()

In [ ]:
epa_9.shape

# CONCILIACION ALL

In [ ]:
txn_epa.columns

In [ ]:
cruce_2 = db.merge(txn_epa[['order_id','total_amount_bruto','fee_amount_mdr','fee_amount_msi','iva_msi','iva_mdr'
                           ,'Date_process_roc','roc_deposit_date']],on='order_id',how='left')
cruce_2['Date_process_roc'].fillna('NO ENCONTRADO',inplace=True)
cruce_2['roc_deposit_date'].fillna('NO ENCONTRADO',inplace=True)
cruce_2['total_amount_bruto'].fillna(0,inplace=True)
cruce_2['fee_amount_mdr'].fillna(0,inplace=True)
cruce_2['fee_amount_msi'].fillna(0,inplace=True)
cruce_2['iva_msi'].fillna(0,inplace=True)
cruce_2['iva_mdr'].fillna(0,inplace=True)
cruce_2.loc[~cruce_2['status_code'].str.contains('3'),'amount_new'] = cruce_2['amount']
cruce_2.loc[cruce_2['status_code'].str.contains('3'),'amount_new'] = cruce_2['amount']*-1
cruce_2['comparison'] = cruce_2['total_amount_bruto'] == cruce_2['amount_new']
cruce_2.head()

In [ ]:
cruce_2.shape

In [ ]:
# cruce_2.to_csv('CLIP-AMEX_junio_2022.csv')

# TRANSACTIONS CLIP - AMEX

In [ ]:
txn_epa_txn_c_a = cruce_2[(cruce_2['date_transaction'] >= start_date) & (cruce_2['date_transaction'] <= final_date)]
txn_epa_txn_c_a.sort_values(by=['date_transaction','order_id'],inplace=True)
txn_epa_txn_c_a.head()

In [ ]:
txn_epa_txn_c_a.shape

In [ ]:
duplicates_2_c_a = txn_epa_txn_c_a[txn_epa_txn_c_a.duplicated(['order_id'] , keep=False)]
duplicates_2_c_a.sort_values(by=['order_id','date_transaction'],inplace=True)
duplicates_2_c_a.head()


In [ ]:
duplicates_2_c_a.shape

In [ ]:
txn_epa_txn_c_a.drop_duplicates(subset ='order_id', keep='last', inplace=True)
txn_epa_txn_c_a.shape

In [ ]:
comparison_2_c_a = txn_epa_txn_c_a[(txn_epa_txn_c_a['comparison'] == False)]
comparison_2_c_a.head()

In [ ]:
comparison_2_c_a.shape

# GROUP BY TRANSACTION DATE

In [ ]:
txn_txn_date_cruce_2_c_a = txn_epa_txn_c_a.groupby(['date_transaction']).agg({'amount':sum,'total_amount_bruto':sum
                                                                              ,'fee_amount_mdr':sum,'fee_amount_msi':sum
                                                                              ,'iva_mdr':sum,'iva_msi':sum,'amount_new':sum
                                                                              ,'order_id':'count'}).reset_index()
txn_txn_date_cruce_2_c_a.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_date_cruce_2_c_a['diff'] = txn_txn_date_cruce_2_c_a['total_amount_bruto'] - txn_txn_date_cruce_2_c_a['amount_new']
txn_txn_date_cruce_2_c_a.head()


In [ ]:
txn_txn_date_cruce_2_c_a.shape

# GROUP BY TRANSACTION DATE AND STATUS CODE

In [ ]:
txn_txn_status_cruce_2_c_a = txn_epa_txn_c_a.groupby(['affiliation_type','date_transaction','status_code']).agg({'amount':sum
                                                                                              ,'total_amount_bruto':sum
                                                                                              ,'fee_amount_mdr':sum
                                                                                              ,'fee_amount_msi':sum
                                                                                              ,'iva_mdr':sum,'iva_msi':sum
                                                                                              ,'amount_new':sum,'order_id':'count'}).reset_index()
txn_txn_status_cruce_2_c_a.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_status_cruce_2_c_a['diff'] = txn_txn_status_cruce_2_c_a['total_amount_bruto'] - txn_txn_status_cruce_2_c_a['amount_new']
txn_txn_status_cruce_2_c_a.head()


In [ ]:
txn_txn_status_cruce_2_c_a.shape

### STATUS 1

In [ ]:
txn_txn_status_cruce_1_status_1_c_a = txn_txn_status_cruce_2_c_a[(txn_txn_status_cruce_2_c_a['status_code'] == '1')]
txn_txn_status_cruce_1_status_1_c_a.head()

In [ ]:
txn_txn_status_cruce_1_status_1_c_a.shape

### STATUS 3

In [ ]:
txn_txn_status_cruce_1_status_3_c_a = txn_txn_status_cruce_2_c_a[(txn_txn_status_cruce_2_c_a['status_code'] == '3')]
txn_txn_status_cruce_1_status_3_c_a.head()

In [ ]:
txn_txn_status_cruce_1_status_3_c_a.shape

# POSIBLES DISPUTAS

In [ ]:
posibles_disputas = txn_epa_txn_c_a[(txn_epa_txn_c_a['status_code'] != '1') & (txn_epa_txn_c_a['status_code'] != '3')]
posibles_disputas.head()


In [ ]:
posibles_disputas.shape

# EXCEL FINAL

In [ ]:
writer = pd.ExcelWriter('CLIP-AMEX JULIO 2021.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
txn_epa_txn_c_a.to_excel(writer, sheet_name='TRANSACTIONS DETAIL',index=False)
duplicates_2_c_a.to_excel(writer, sheet_name='DUPLICATES',index=False)
comparison_2_c_a.to_excel(writer, sheet_name='TRANSACTIONS COMPARISON',index=False)
txn_txn_date_cruce_2_c_a.to_excel(writer, sheet_name='TXN DATE',index=False)
txn_txn_status_cruce_2_c_a.to_excel(writer, sheet_name='TXN DATE AND STATUS',index=False)
txn_txn_status_cruce_1_status_1_c_a.to_excel(writer, sheet_name='STATUS 1',index=False)
txn_txn_status_cruce_1_status_3_c_a.to_excel(writer, sheet_name='STATUS 3',index=False)
posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS',index=False)
writer.save()
writer.close()